<a href="https://colab.research.google.com/github/emily1618/Data-Portfolio/blob/main/Best_Day_To_Invest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np



In [5]:
# read the data

data_import = pd.read_csv('/content/VOO.csv', parse_dates=['Date'])
data_import

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-09-09,102.500000,102.500000,101.139999,101.320000,81.350807,26500
1,2010-09-10,101.680000,101.860001,101.300003,101.779999,81.720131,8600
2,2010-09-13,102.959999,103.139999,102.500000,103.059998,82.747894,33750
3,2010-09-14,102.839996,103.480003,102.379997,103.040001,82.731789,59400
4,2010-09-15,102.620003,103.379997,102.400002,103.300003,82.940575,9250
...,...,...,...,...,...,...,...
2876,2022-02-10,414.929993,420.799988,411.140015,413.179993,413.179993,9260500
2877,2022-02-11,413.149994,415.200012,403.570007,404.940002,404.940002,10452700
2878,2022-02-14,404.429993,405.989990,400.239990,403.619995,403.619995,10162000
2879,2022-02-15,408.179993,410.290009,407.459991,410.100006,410.100006,6822300


In [8]:
# add a date of week

day_mapper = {0: "Monday", 1:"Tuesday", 2:"Wednesday", 3:"Thursday", 4:"Friday", 5:"Saturday", 6:"Sunday"}
data_import["DayOfWeek"] = data_import["Date"].map(lambda x: day_mapper[x.dayofweek])
data_import

,Date,Open,High,Low,Close,Adj Close,Volume,DayOfWeek
0,2010-09-09,102.500000,102.500000,101.139999,101.320000,81.350807,26500,Thursday
1,2010-09-10,101.680000,101.860001,101.300003,101.779999,81.720131,8600,Friday
2,2010-09-13,102.959999,103.139999,102.500000,103.059998,82.747894,33750,Monday
3,2010-09-14,102.839996,103.480003,102.379997,103.040001,82.731789,59400,Tuesday
4,2010-09-15,102.620003,103.379997,102.400002,103.300003,82.940575,9250,Wednesday
...,...,...,...,...,...,...,...,...
2876,2022-02-10,414.929993,420.799988,411.140015,413.179993,413.179993,9260500,Thursday
2877,2022-02-11,413.149994,415.200012,403.570007,404.940002,404.940002,10452700,Friday
2878,2022-02-14,404.429993,405.989990,400.239990,403.619995,403.619995,10162000,Monday
2879,2022-02-15,408.179993,410.290009,407.459991,410.100006,410.100006,6822300,Tuesday


## Average Dip Per Day

In [9]:
# current value - previous value = difference

data_import["diff_from_previous_day"] = (data_import["Open"].diff()/data_import["Open"]) * 100

In [10]:
# group by the day of the week and check the mean of the diff from previous day in the last 12 years
# biggest the avg dip is a Monday (google search the "weekend effect")

data_import.groupby("DayOfWeek")["diff_from_previous_day"].mean()

DayOfWeek
Friday       0.064210
Monday      -0.034693
Thursday    -0.001912
Tuesday      0.103887
Wednesday    0.076498
Name: diff_from_previous_day, dtype: float64

## Checking the Share Method

In [15]:
# create a range of dates 

dates = pd.date_range(start=data_import["Date"].min(), end=data_import["Date"].max())


In [17]:
# create a table just the date and the day of the week

date_table = pd.DataFrame(data={"Date":dates})
date_table["Weekday"] = date_table["Date"].map(lambda x: day_mapper[x.dayofweek])
date_table

,Date,Weekday
0,2010-09-09,Thursday
1,2010-09-10,Friday
2,2010-09-11,Saturday
3,2010-09-12,Sunday
4,2010-09-13,Monday
...,...,...
4174,2022-02-12,Saturday
4175,2022-02-13,Sunday
4176,2022-02-14,Monday
4177,2022-02-15,Tuesday


In [37]:
# Join the date table to the data_import table

full_calendar = pd.merge(left=date_table, right=data_import, how='left', left_on='Date', right_on='Date')

In [38]:
# the date_table include Saturdays and Sundays, but there are no trading days. So we need to take off the weekends.
# if the data is in the conditions, using ~ get rid of those data

full_calendar = full_calendar[~full_calendar["Weekday"].isin(["Saturday","Sunday"])]




In [40]:
full_calendar.rename(columns={"Date":"Trading Day"}, inplace=True)
full_calendar

,Trading Day,Weekday,Open,High,Low,Close,Adj Close,Volume,DayOfWeek,diff_from_previous_day
0,2010-09-09,Thursday,102.500000,102.500000,101.139999,101.320000,81.350807,26500.0,Thursday,NaN
1,2010-09-10,Friday,101.680000,101.860001,101.300003,101.779999,81.720131,8600.0,Friday,-0.806452
4,2010-09-13,Monday,102.959999,103.139999,102.500000,103.059998,82.747894,33750.0,Monday,1.243200
5,2010-09-14,Tuesday,102.839996,103.480003,102.379997,103.040001,82.731789,59400.0,Tuesday,-0.116689
6,2010-09-15,Wednesday,102.620003,103.379997,102.400002,103.300003,82.940575,9250.0,Wednesday,-0.214376
...,...,...,...,...,...,...,...,...,...,...
4172,2022-02-10,Thursday,414.929993,420.799988,411.140015,413.179993,413.179993,9260500.0,Thursday,-0.872435
4173,2022-02-11,Friday,413.149994,415.200012,403.570007,404.940002,404.940002,10452700.0,Friday,-0.430836
4176,2022-02-14,Monday,404.429993,405.989990,400.239990,403.619995,403.619995,10162000.0,Monday,-2.156121
4177,2022-02-15,Tuesday,408.179993,410.290009,407.459991,410.100006,410.100006,6822300.0,Tuesday,0.918712


In [45]:
# use the following copy/paste the dataframe to check in excel

#full_calendar.to_clipboard()

In [44]:
# backfill the days with trade where there is no trading activity and it's a holiday, not weekend such as Thanksgiving day
# take the next value and pull it up

full_calendar = full_calendar.bfill(axis='rows').reset_index(drop=True)




In [46]:
# test the weekday to make sure all trading days are consistent

full_calendar['Weekday'].value_counts()

Thursday     597
Friday       597
Monday       597
Tuesday      597
Wednesday    597
Name: Weekday, dtype: int64

In [52]:
# create a function that loop through Monday to Friday (all available trading days) and simulate how much money one can earn if putting in $100 a week
# temp data is to filter down to one of day of the week

def trading_test(day, amount_to_invest, data):
  temp_data = data[data["Weekday"]==day]
  temp_data["Shares Owned"] = amount_to_invest/temp_data["Open"]
  final_price = temp_data['Open'].iloc[-1] #last open price of the data
  final_amount = temp_data["Shares Owned"].sum() * final_price #value of the portofolio
  return final_amount


In [53]:
full_calendar["Weekday"].unique()

array(['Thursday', 'Friday', 'Monday', 'Tuesday', 'Wednesday'],
      dtype=object)

In [54]:
# Using this method, it looks like invest recurringly on a Thursday will get you the most money over time

for i in full_calendar["Weekday"].unique():
  print(i, trading_test(i, 100, full_calendar))

Thursday 131620.1405065215
Friday 130975.84906708827
Monday 128213.72082537432
Tuesday 129321.07678847888
Wednesday 129203.71831904375


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## Facebook Prophet Method

In [ ]:
def prophet_predictor(dataset):
    # needs to take 3 weeks of data (monday - friday)
    # predict the 4th week
    # say which day has the lowest value per week (defined by Monday)
    # compare that to actual data
    # Accuacy is our metric

    # Need to consider Holidays in the future
    m = Prophet()

    m.fit(dataset)

    future = m.make_future_dataframe(periods = 7)

    forecast = m.predict(future)
    return forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
forecast_data = []

# Start at 17
# First iteration got to 1042 -> start second at 281 (for some reason the indexing doesn't work as expected)
# Third iteration needs to start at 2007 which is index 402

for i in full_calendar[full_calendar["Weekday"] == "Monday"]["Calendar Date"].iloc[402:].index:
    days_to_include = i - 15
    temp = full_calendar.iloc[days_to_include:i, [0, 3]]
    temp.rename(columns={"Calendar Date":"ds", "Open":"y"}, inplace=True)
    prophet_predictor(temp).to_pickle(f"three_week_forecast//{i}.pkl")
    # forecast_data.append(prophet_predictor(temp)) # The for loop kept seizing up for some reason
    clear_output(wait=True) # Stops the output from getting too long
    print(i, "/", full_calendar[full_calendar["Weekday"] == "Monday"]["Calendar Date"].iloc[17:].index.max())

In [ ]:
# Import and append all of the Pickle files

files_to_append = []

files_to_read = ["three_week_forecast//" + i for i in os.listdir("three_week_forecast")]

for i in files_to_read:
    files_to_append.append(pd.read_pickle(i))

In [ ]:
# Combine all of the saved Pickle files

predicted_values = pd.concat(files_to_append).sort_values("ds")
predicted_values.rename(columns={"ds":"Predicted_Date", "yhat":"Prediction"}, inplace=True)
predicted_values

In [ ]:
# The lowest value day per week: Predicted Part 1

full_calendar["Week Monday"] = full_calendar["Calendar Date"] - pd.to_timedelta(full_calendar["Calendar Date"].dt.dayofweek, unit='d')

predicted_values["Week Monday"] = predicted_values["Predicted_Date"] - pd.to_timedelta(predicted_values["Predicted_Date"].dt.dayofweek, unit='d')

min_predicted_values = predicted_values.groupby("Week Monday")["Prediction"].min().reset_index()
min_predicted_values

In [ ]:
# The lowest value day per week: Predicted Part 2

predicted_values_output = pd.merge(min_predicted_values, predicted_values, how="left", on=["Week Monday", "Prediction"])
predicted_values_output = predicted_values_output.groupby("Week Monday")["Predicted_Date"].min().reset_index()
predicted_values_output["Predicted_Min_Day"] =  predicted_values_output["Predicted_Date"].map(lambda x: day_mapper[x.dayofweek])
predicted_values_output

In [ ]:
# The lowest value day per week: Actual

min_values_per_week = full_calendar.groupby("Week Monday")["Open"].min().reset_index()
actual_values_min_per_week = pd.merge(min_values_per_week, full_calendar, how="left", on=["Week Monday", "Open"])
actual_values_min_per_week = actual_values_min_per_week.groupby("Week Monday")["DayOfWeek"].min().reset_index()
actual_values_min_per_week.rename(columns={"DayOfWeek":"Actual Min Day"}, inplace=True)
actual_values_min_per_week

In [ ]:
# Combine the actual and predicted min days per week

combined_prophet_predictions = pd.merge(actual_values_min_per_week, predicted_values_output, how="left", on="Week Monday")
combined_prophet_predictions = combined_prophet_predictions.dropna()
combined_prophet_predictions["Correct Predictions"] = combined_prophet_predictions.apply(lambda x: 1 if x["Actual Min Day"] == x["Predicted_Min_Day"] else 0, axis='columns')
combined_prophet_predictions

In [ ]:
# Calculate the percent of accurate calculations

combined_prophet_predictions["Correct Predictions"].sum() / len(combined_prophet_predictions)